# Introduction

# Model Architecture
- 지금까지 Encoder-Decoder 구조의 인공 시퀀스 전이 모델이 가장 효과적이었다 
- encoder map: x로 표현되는 input sequence가 인코더 맵을 통해 continuous representation z로 변환됨
- z가 주어지면 Decoder는 한번에 하나씩의 output sequence y를 만들어냄
  : z는 query * key / 8 -> softmax -> * value 한 값 
- 각 스텝마다 모델은 auto-regressive를 사용하고 다음 값을 만들어내기 위해 이전에 생성된 값을 additional input으로 사용한다 
- Transformer는 위의 과정을 Encoder와 Decoder 내에서 self-attention, point-wise, fully-connected layer를 stacking하여 진행한다 (Fig1의 왼쪽 부분)
  
## 3-1. Encoder and Decoder Stacks
### Encoder 
- Encoder는 6개의 동일한 layer로 이루어져있으며, 각 레이어는 2개의 sub-layer로 이루어져있다
- 첫번째 레이어는 Multi-head self Attention이며, 두번째 레이어는 단순한 position-wise fully connected feed forward network이다
- 2개의 sub-layer의 주변을 residual-connection을 사용하고, 뒤이어 Layer Normalization을 사용한다 
- 즉, 각 sublayer에서 나오는 output은 LayerNorm(x + Sublayer(x))이다
- Sublayer(x)는 sub-layer 그 자체를 함수에 적용한 값이다
- 해당 residual connections들을 사용하려면, 해당 모델의 모든 sublayer들은 embedding layer여야하며, output의 dimension은 512로 나와야 한다($d_model = 512$)

### Decoder 
- Decoder 또한 6개의 동일한 레이어로 구성되어있다
- Decoder는 인코더 레이어와 같이 2개의 sub-layer는 동일하지만 여기에 인코더 스택에서 나오는 output을 사용하는 multi head attention 레이어를 3번째 sublayer로써 추가하였다(Encoder-Decoder Attention: Multihead와 Feed forward 사이에 끼어있음)
- 인코더와 동일하게 각 sublayer들의 주변에 residual connection을 사용했고 뒤이어 Layer Normalization을 진행한다
- Decoder stack에 있는 self attention sub-layer는 다음 포지션에 적용하기 위한 포지션을 보존하기 위해 수정을 진행한다 (Masked Multi head attention)
- 이는 Masking이라 하며, output embedding이 한 포지션에 의해 상쇄된다는 점과, i 포지션을 예측하는 것이 i 보다 낮은 포지션에 있는 output들에 의존된다는 것이 종합된 결과이다

## 3-2. Attention
- attention function은 query와 output의 key-value 값을 mapping하는 과정이다 (q, k, v는 다 vector)
- output은 query와 그에 맞는 key값의 적합성을 계산한 value값에 weight를 주어 나온 weighted sum of values이다

### 추가 설명 
Q : 영향을 받는 단어 
K : 영향을 주는 단어 
V : 영향에 대한 가중치 
- Q와 K의 Dot Product은 영향을 주는 단어와 받는 단어의 유사도를 측정하는 것 

### 3-2-1. Scaled Dot-Product Attention
- Scaled Dot Product Attention의 input은 $d_k$의 dimension을 가진 query와 key들로 이루어져 있다 
- 모든 query와 key의 dot products를 계산하고, 이를 $sqrt(d_k)$로 나눈다음 softmax를 통과시켜 각 value에 대한 weight를 얻는다
- query 계산은 동시다발적으로 이루어지며, 이는 matrix Q로 통합되어 있다 
- key와 value 또한 matrix K와 V로 통합되어 있다 
- output 행렬의 결과 수식은 아래와 같다 
- $Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) *V$
- attention function으로 가장 많이 사용되는 것은 additive attention과 dot-product(multiplicative) attention이다 
- Dot-product attention은 scaling factor ($1/sqrt(d_k)$)를 제외하고는 우리의 알고리즘과 동일하다 
- Additive Attention은 한개의 hidden layer와 feed-forward network를 사용하여 적합성을 계산한다 
- Dot-Product 및 Additive attention은 이론적으로는 비슷하지만 현실에서는 Dot-product attention이 (더 효율적인 mtrix multiplication 코드를 사용하기 때문에)훨씬 더 빠르고 공간을 효율적으로 사용하는 방법이다. 
- 작은 value $d_k$에서는 2 mechanism이 비슷하게 수행되지만 큰 값의 $d_k$에서는 additive attention이 훨신 좋은 결과를 보인다 
- 이는 dot-product으로 큰 value의 $d_k$를 계산하면 큰 규모의 결과값이 나오기 때문에 softmax를 통과시켰을 때 gradient가 심히 작아진다는 문제가 있어서 생기는 결과로 보인다 
- 그렇기 떄문에 Scaling $1/sqrt(d_k)$를 수행한다

-> 그니까 위와 같이 작용하는 2가지 종류 attention 중에서 Dot product를 쓴 이유는 계산이 빠르기 때문이고, Gradient Descent의 단점을 없애기 위해 scaling 진행했다는 것

### 3-2-2. Multi-head Attention
- $d_model$ dimension의 q, k, v를 사용하는 것보다 q, k, v를 h번 (각 횟수마다 다른 식으로) linear하게 $d_k, d_v$를 사용하는 것이 더 효과적인 결과를 내는 것을 확인했다. 
- 투영된 버전의 q, k, v를 가지고 병렬적으로 attention function을 진행하여 $d_v$ dimension의 output value를 얻어낸다 
- 그리고 이를 합쳐서 다시 한번 투영해내면 final value를 얻는다 
- Multi head attention은 다른 위치의 다른 공간을 대표하는 정보들을 한번에 모아준다 
- single attention head로 진행할 경우 해당 결과를 억제한다(단어를 바꿔야겠군 ㅇㅅㅇ)
- 해당 작업은 8개의 병렬 attention layer 혹은 head를 사용한다
- 그리고 $d_k = d_v = d_model / h = 64$ 로 사용한다 
- 해당 작업은 각 head의 dimension을 줄임으로써 full dimension의 single head attention과 같이 계산 비용을 줄이기 위해 진행한다 

### 3-2-3. Applications of Attention in our Model 
Transformer는 Multihead attention을 3가지 방법으로 사용한다 
- Encoder-Decoder Attention Layer에서, query는 이전 디코더 레이어에서 제공되며 memory key, value는 encoder의 output 값이다. 이는 Decoder에 있는 모든 포지션이 input sequence의 모든 포지션에 적용될 수 있게한다. 이는 전형적인 sequence-to-sequence 모델의 Encoder-Decoder attention mechanism과 같은 역할을 한다 
- Self Attention layer를 갖고있는 Encoder. Self attention layer 이전의 레이어가 Encoder면 self attention layer의 모든 key, value, query는 동일한 공간에서 나온다. Encoder에서의 각 포지션은 인코더 이전의 레이어에서의 모든 포지션을 처리할 수 있다 
- Decoder의 self-attention layer도 Decoder의 각 포지션이 Decoder에 있는 모든 포지션까지 처리할 수 있도록 한다. 우리는 auto regressive property를 보존하기 위해서 Decoder의 왼쪽으로 치우친 정보 흐름을 막아야 한다. 따라서 scaled dot product attention 안에서 softmax를 통과하는 모든 value에 대해 masking을 진행했다. (Fig 2)


### 3-3. Position-wise Feed-Forward Networks
- attention 각 sub-layer들마다 Encoder, Decoder는 각 포지션에 독립적으로 적용되는 fully connected feed forward network를 갖고 있다 
- 이는 사이에 ReLU 활성화 함수를 가진 linear transformation을 구성한다 
- 다른 포지션을 가진 동일한 linear transformation은 레이어와 레이어 사이에서 다른 parameter를 갖는다 
- 이를 다른 말로 설명하자면 kernel size 1을 갖는 2개의 Conv layer이다. 
- input과 output의 dimension은 512이고, inner-layer는 2048의 dimension이다 

### 3-4. Embeddings and Softmax
- 다른 시퀀스 전이 모델과 같이 우리는 input token과 Output token을 $d_model$의 dimension을 가진 vector로 변환하려 할 때 embedding을 사용한다 
- 우리는 또한 linear transformation과 softmax function을 이용하여 decoder output으로 다음 토큰의 가능성을 예측한다 
- 우리의 모델에서는 2개의 embedding layer와 pre-softmax linear transformation 사이에서 동일한 weight matrix를 사용한다 
- embedding layer에서는 해당 weight들에 $sqrt(d_model)$을 곱해준다 

#### 왜 해당 값을 곱하지?
1. Answer1: The reason we increase the embedding values before the addition is to make the positional encoding relatively smaller. This means the original meaning in the embedding vector won’t be lost when we add them together.

2. https://stats.stackexchange.com/questions/534618/why-are-the-embeddings-of-tokens-multiplied-by-sqrt-d-note-not-divided-by-sq 
-> embedding할 때 

### 3-5. Positional Encoding
- 우리 모델이 reccurence, convolusion을 갖고있지 않지만 시퀀스의 순서를 사용하기 위해서는 시퀀스 토큰의 절대적 위치나 이와 관련된 정보를 추가시켜줘야 한다 
- 그래서 Encoder와 Decoder stack의 맨 아래의 input embedding에 'positional encoding'을 추가한다 
- positional encoding은 embedding과 같이 $d_model$의 dimension을 갖고 있고, 그래서 두 값을 더할 수 있다
- positional encoding을 할 수 있는 방법은 다양하다 
- 여기서는 sine과 cosine 함수의 다른 frequency를 사용하겠다 
- $PE(pos,2i) =sin(pos/10000^(2i/dmodel))$
- $PE(pos,2i+1) =cos(pos/10000^(2i/dmodel))$
- pos는 position이고, i는 dimension을 의미한다 
- 즉, 각 positional encoding의 각 dimension은 sinusoid(sine 곡선_$2pi$에서 $10000*2pi$ 사이의 등비수열 파장 모양)와 응답한다 
- 우리는 해당 모양이 모델이 각 관련 포지션을 잘 배울 수 있게 할 것이라 가정햤기 떄문에 해당 함수를 선택했다(고정된 오프셋 k, $PE_(pos+k)$가 $PE_pos$의 linear function을 잘 표현하기 때문에)
- 또한 학습된 positional embedding을 사용하는것을 실험했고, 2개 버전의 아주 비슷한 결과를 얻어냈다 (Table 3, row E)
- 우리는 sinusoidal version을 선정했고, 그 이유는 해당 모델이 학습중에 시퀀스 길이가 길더라도 추론할 수 있을 것이라 생각했기 때문이다 

## 5. Training 

### 5-1. Trainig Data and Batching
- 450만 시퀀스 쌍을 갖고있는 standard WMT 2014 English-German dataset으로 학습했다 
- 문장은 byte-pair encoding되어있었고, source target 단어는 37000개의 토큰을 공유하고 있었다 
- 영문 to 불어를 위해서는 상대적으로 큰 규모의 3600만 문장으로 이루어지고 32000개의 분리된 token(단어)를 갖고있는 larger WMT 2014 English-French dataset를 사용했다 
- Sentence pair는 대략적 시퀀스 길이에 맞춰서 배치되었다 
- 각 학습 배치는 